Here's a simple Gurobi implementation to solve an inventory management problem using Dynamic Programming (DP). The goal is to determine the optimal order quantity for each period to minimize total costs, which include holding costs, shortage costs, and ordering costs.

Problem setup:

We will minimize the total cost over a number of periods.
Costs: Holding cost, shortage cost, and ordering cost.
Constraints: Non-negative inventory, demand in each period.

In [2]:
import gurobipy as gp
from gurobipy import GRB

# Data
periods = 12  # number of periods (e.g., 12 months)
demand = [50, 30, 40, 20, 60, 80, 30, 50, 70, 90, 30, 40]  # demand for each period
holding_cost = 2  # cost per unit held in inventory
shortage_cost = 5  # cost per unit of shortage
ordering_cost = 50  # fixed cost per order
max_inventory = 100  # maximum inventory capacity
big_M = 1000  # A large number for binary constraints

# Model
model = gp.Model('Inventory_Optimization')

# Variables
order_qty = model.addVars(periods, vtype=GRB.CONTINUOUS, name='order_qty')  # Order quantity in each period
inventory = model.addVars(periods, vtype=GRB.CONTINUOUS, lb=0, name='inventory')  # Inventory at the end of each period
shortage = model.addVars(periods, vtype=GRB.CONTINUOUS, lb=0, name='shortage')  # Shortage in each period
order_decision = model.addVars(periods, vtype=GRB.BINARY, name='order_decision')  # Binary decision if ordering occurs

# Initial inventory (set to zero for simplicity)
initial_inventory = 0

# Objective: Minimize total cost (holding cost + shortage cost + ordering cost)
model.setObjective(
    gp.quicksum(holding_cost * inventory[t] + shortage_cost * shortage[t] + ordering_cost * order_decision[t] for t in range(periods)),
    GRB.MINIMIZE
)

# Constraints
for t in range(periods):
    if t == 0:
        # Inventory balance for first period
        model.addConstr(inventory[t] == initial_inventory + order_qty[t] - demand[t] + shortage[t], name=f'bal_{t}')
    else:
        # Inventory balance for subsequent periods
        model.addConstr(inventory[t] == inventory[t - 1] + order_qty[t] - demand[t] + shortage[t], name=f'bal_{t}')

    # Ensure no ordering if no order decision
    model.addConstr(order_qty[t] <= big_M * order_decision[t], name=f'order_decision_{t}')

    # Inventory must be non-negative and less than or equal to max capacity
    model.addConstr(inventory[t] >= 0, name=f'non_negative_inventory_{t}')
    model.addConstr(inventory[t] <= max_inventory, name=f'max_inventory_{t}')

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    for t in range(periods):
        print(f"Period {t + 1}: Order Quantity = {order_qty[t].x:.2f}, Inventory = {inventory[t].x:.2f}, Shortage = {shortage[t].x:.2f}, Order Decision = {order_decision[t].x:.0f}, demand = {demand[t]}")
else:
    print('No optimal solution found')


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 48 rows, 48 columns and 95 nonzeros
Model fingerprint: 0x304e7917
Variable types: 36 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+02]
Found heuristic solution: objective 2950.0000000
Presolve removed 24 rows and 12 columns
Presolve time: 0.00s
Presolved: 24 rows, 36 columns, 60 nonzeros
Variable types: 24 continuous, 12 integer (12 binary)

Root relaxation: objective 2.234315e+02, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   